
# 采用变分算法重构普适双比特门

尝试采用两个 CNOT 门构建，发现这是不可以的。

变分线路构建

In [18]:
import numpy as np
from mindquantum import *
from scipy.linalg import expm
import mindspore as ms
from mindspore import ops
ms.context.set_context(mode=ms.context.PYNATIVE_MODE, device_target="CPU")
from mindspore.nn import Adam, TrainOneStepCell
from mindspore.common.parameter import Parameter
from mindspore.common.initializer import initializer  
ms.set_seed(1)
np.random.seed(1)

train_x = np.load('./src/2_qubit_train_x.npy', allow_pickle=True)
eval_x = np.load('./src/2_qubit_eval_x.npy', allow_pickle=True)
train_y = np.load('./src/2_qubit_train_y.npy', allow_pickle=True)
eval_y = np.load('./src/2_qubit_eval_y.npy', allow_pickle=True)
u_mats = np.load('./src/2_qubit_u.npy', allow_pickle=True)

s_x = X.matrix()
s_z = Z.matrix()

circ = Circuit()
circ += U3('00','01','02', 0)
circ += U3('10','11','12', 1)
circ += X.on(1,0)
circ += U3('03','04','05', 0)
circ += U3('13','14','15', 1)
circ += X.on(0,1)
circ += U3('06','07','08', 0)
circ += U3('16','17','18', 1)
# circ += X.on(1,0)
# circ += U3('09','010','011', 0)
# circ += U3('19','110','111', 1)

ham = Hamiltonian(QubitOperator('')) 
sim = Simulator('mqvector', circ.n_qubits)
sim_left = Simulator('mqvector',circ.n_qubits)

grad_ops = sim.get_expectation_with_grad(ham,
                                         circ,
                                         circ_left=Circuit(),
                                         simulator_left=sim_left,
                                         ansatz_params_name=circ.params_name)

class MQLayer(ms.nn.Cell):
    def __init__(self, expectation_with_grad, weight='normal'):
        super(MQLayer, self).__init__()
        self.evolution = MQN2Ops(expectation_with_grad)
        weight_size = len(self.evolution.expectation_with_grad.ansatz_params_name)
        self.weight = Parameter(initializer(weight,
                                            weight_size,
                                            dtype=ms.float32),
                                name='ansatz_weight')

    def construct(self, x, y):
        return self.evolution(self.weight, x, y)
    

acc_list = []
for i in range(len(u_mats)):
    QuantumNet = MQLayer(grad_ops)
    opti = Adam(QuantumNet.trainable_params(), learning_rate=0.02)     
    net = TrainOneStepCell(QuantumNet, opti)
    
    for j in range(len(train_x)):
        res = net(ms.Tensor(train_x[j]), ms.Tensor(train_y[i,j]))
        
    final_state = []
    for j in range(len(eval_x)):
        sim.reset()
        sim.set_qs(eval_x[j])
        sim.apply_circuit(circ, QuantumNet.weight.asnumpy())
        final_state.append(sim.get_qs())
        
    acc = np.real(np.mean([np.abs(np.vdot(bra, ket)) for bra, ket in zip(np.array(final_state), eval_y[i])]))
    acc_list.append(acc)
    print(f'{i+1}/{len(u_mats)}', '  acc:', acc)
    
print('\n平均保真度为：', np.mean(acc_list))

1/10   acc: 0.9975399483153466
2/10   acc: 0.9859606641409581
3/10   acc: 0.9970366827132137
4/10   acc: 0.9466270623847457
5/10   acc: 0.9994909995547852
6/10   acc: 0.9513061070991679
7/10   acc: 0.9838905556590538
8/10   acc: 0.9945475024451836
9/10   acc: 0.9397806431018944
10/10   acc: 0.9994736313386009

平均保真度为： 0.979565379675295
